## Demo for user story 105

The following scenario demonstrates the implementation of user story 105 for the RS Server Catalog.

The goal of this user story is to add a new search endpoint that can only search inside a collection,
so that the user can search easily for STAC items in a specific collections.

The following endpoints may now be called with an optional usage of the OwnerId parameter:
* GET /catalog/collections/{ownerId:collectionId}/search
* POST /catalog/collections/{ownerId:collectionId}/search

There is also a new fonction in the stac_client called "search_inside_collection" that can only search inside a collection.

### Initialisation

In [1]:
# Init environment before running a demo notebook.
from resources.utils import *
init_demo()
from resources.utils import * # reload the global vars again

I0000 00:00:1720704403.447065     110 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache, work_serializer_dispatch
I0000 00:00:1720704403.447220     110 ev_epoll1_linux.cc:125] grpc epoll fd: 59
Overriding of current TracerProvider is not allowed
Attempting to instrument while already instrumented


Auxip service: http://rs-server-adgs:8000
CADIP service: http://rs-server-cadip:8000
Catalog service: http://rs-server-catalog:8000


### Create a collection and 5 items to serve as the base for the demo

In [2]:
# Create a test collection and stage test items
collection = create_test_collection()
items = stage_test_several_items()

Staging status for 'DCS_01_S1A_20200105072204051312_ch1_DSDB_00004.raw': IN_PROGRESS
Staging status for 'DCS_01_S1A_20200105072204051312_ch1_DSDB_00004.raw': DONE


Staging status for 'DCS_01_S1A_20200105072204051312_ch1_DSDB_00003.raw': IN_PROGRESS
Staging status for 'DCS_01_S1A_20200105072204051312_ch1_DSDB_00003.raw': DONE


Staging status for 'DCS_01_S1A_20200105072204051312_ch1_DSDB_00002.raw': IN_PROGRESS
Staging status for 'DCS_01_S1A_20200105072204051312_ch1_DSDB_00002.raw': DONE


Staging status for 'DCS_01_S1A_20200105072204051312_ch1_DSDB_00001.raw': IN_PROGRESS
Staging status for 'DCS_01_S1A_20200105072204051312_ch1_DSDB_00001.raw': DONE


Staging status for 'DCS_01_S1A_20200105072204051312_ch1_DSDB_00000.raw': IN_PROGRESS
Staging status for 'DCS_01_S1A_20200105072204051312_ch1_DSDB_00000.raw': DONE




In [19]:
import requests

HEADERS={"headers": {"x-api-key": apikey}}

Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED

### Calling The endpoint GET /catalog/collections/{ownerId:collectionId}/search

In [4]:
import json

test_params = {"filter": "width=2500"}

response = requests.get(f"{stac_client.href_catalog}/catalog/collections/jovyan:{collection.id}/search", params=test_params, **HEADERS)
response.raise_for_status()

content = json.loads(response.content)
content

{'type': 'FeatureCollection',
 'context': {'limit': 10, 'returned': 3},
 'features': [{'id': 'DCS_01_S1A_20200105072204051312_ch1_DSDB_00000.raw',
   'bbox': [-180.0, -90.0, 180.0, 90.0],
   'type': 'Feature',
   'links': [{'rel': 'collection',
     'type': 'application/json',
     'href': 'http://rs-server-catalog:8000/collections/jovyan_my_test_collection'},
    {'rel': 'parent',
     'type': 'application/json',
     'href': 'http://rs-server-catalog:8000/collections/jovyan_my_test_collection'},
    {'rel': 'root',
     'type': 'application/json',
     'href': 'http://rs-server-catalog:8000/'},
    {'rel': 'self',
     'type': 'application/geo+json',
     'href': 'http://rs-server-catalog:8000/collections/jovyan_my_test_collection/items/DCS_01_S1A_20200105072204051312_ch1_DSDB_00000.raw'}],
   'assets': {'file': {'href': 'https://rs-server-catalog:8000/catalog/collections/jovyan:my_test_collection/items/DCS_01_S1A_20200105072204051312_ch1_DSDB_00000.raw/download/file',
     'alternat

### Calling the endpoint POST /catalog/collections/{ownerId:collectionId}/search

In [22]:
WIDTH = 2500
OWNER = "jovyan"

parameters = {
            "collections": ["my_test_collection"],
            "filter-lang": "cql2-json",
            "filter": {
                "op": "and",
                "args": [
                    {"op": "=", "args": [{"property": "owner"}, OWNER]},
                    {"op": "=", "args": [{"property": "width"}, WIDTH]},
                ],
            },
        }

response = requests.post(f"{stac_client.href_catalog}/catalog/collections/jovyan:{collection.id}/search", json=parameters, **HEADERS)
response.raise_for_status()

content = json.loads(response.content)
content

{'type': 'FeatureCollection',
 'context': {'limit': 10, 'returned': 3},
 'features': [{'id': 'DCS_01_S1A_20200105072204051312_ch1_DSDB_00000.raw',
   'bbox': [-180.0, -90.0, 180.0, 90.0],
   'type': 'Feature',
   'links': [{'rel': 'collection',
     'type': 'application/json',
     'href': 'http://rs-server-catalog:8000/catalog/collections/jovyan:my_test_collection'},
    {'rel': 'parent',
     'type': 'application/json',
     'href': 'http://rs-server-catalog:8000/catalog/collections/jovyan:my_test_collection'},
    {'rel': 'root',
     'type': 'application/json',
     'href': 'http://rs-server-catalog:8000/catalog/catalogs/jovyan'},
    {'rel': 'self',
     'type': 'application/geo+json',
     'href': 'http://rs-server-catalog:8000/catalog/collections/jovyan:my_test_collection/items/DCS_01_S1A_20200105072204051312_ch1_DSDB_00000.raw'}],
   'assets': {'file': {'href': 'https://rs-server-catalog:8000/catalog/collections/jovyan:my_test_collection/items/DCS_01_S1A_20200105072204051312_ch

Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED


### Using the new function "search_inside_collection" from the stac_client

In [23]:
WIDTH = 2500
HEIGHT = 3000

search = stac_client.search_inside_collection(owner_id="jovyan", collection_id="my_test_collection", ids="DCS_01_S1A_20200105072204051312_ch1_DSDB_00002.raw")
results = list(search.items_as_dicts())
print(f"\nFound {len(results)} results")
for result in results:
    print(f"({result['collection']}) {result['id']}")




Found 1 results
(jovyan_my_test_collection) DCS_01_S1A_20200105072204051312_ch1_DSDB_00002.raw


Failed to export metrics to tempo:4317, error code: StatusCode.UNIMPLEMENTED
